In [2]:
!pip install selenium


  Using cached https://files.pythonhosted.org/packages/8d/59/b4572118e098ac8e46e399a1dd0f2d85403ce8bbaad9ec79373ed6badaf9/PySocks-1.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/46/9cb0e58b2deb7f82b84065f37f3bffeb12413f947f9388e4cac22c4621ce/sortedcontainers-2.4.0-py2.py3-none-any.whl


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Inisialisasi WebDriver (pastikan Anda telah menginstal Selenium)
driver = webdriver.Chrome()  # Ganti dengan driver yang sesuai (misalnya, Firefox)

# URL halaman web yang akan di-scraper
url = 'https://bpjt.pu.go.id/tabel-tarif-tol'

# Buka halaman web
driver.get(url)

# Klik tombol "Lihat Tarif Tol"
get_image_button = driver.find_element(By.ID, 'get-image')
get_image_button.click()

# Tunggu hingga tabel terisi (gunakan waktu tunggu jika diperlukan)
wait = WebDriverWait(driver, 10)
table = wait.until(EC.presence_of_element_located((By.ID, 'tariffs-table')))

# Dapatkan isi tabel menggunakan BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
table_data = soup.find('table', {'id': 'tariffs-table'})

# Buat list kosong untuk menyimpan data
data = []

# Loop melalui baris dalam elemen <tbody> tabel
for row in table_data.find('tbody').find_all('tr'):
    asal = row.find_all('td')[0].text
    tujuan = row.find_all('td')[1].text
    golongan = [td.text for td in row.find_all('td')[2:]]
    data.append([asal, tujuan] + golongan)

# Simpan data ke dalam DataFrame
df = pd.DataFrame(data, columns=['Asal Perjalanan', 'Tujuan Perjalanan', 'Golongan I', 'Golongan II', 'Golongan III', 'Golongan IV', 'Golongan V', 'Golongan VI'])

# Cetak DataFrame
print(df)

# Tutup WebDriver
driver.quit()


                                     Asal Perjalanan  \
0  Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...   

                                   Tujuan Perjalanan Golongan I Golongan II  \
0  Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...      16000       23500   

  Golongan III Golongan IV Golongan V Golongan VI  
0        23500       31500      31500           -  


In [6]:
df.head()

,Asal Perjalanan,Tujuan Perjalanan,Golongan I,Golongan II,Golongan III,Golongan IV,Golongan V,Golongan VI
0,"Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...","Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...",16000,23500,23500,31500,31500,-


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Inisialisasi WebDriver (pastikan Anda telah menginstal Selenium)
driver = webdriver.Chrome()  # Ganti dengan driver yang sesuai (misalnya, Firefox)

# URL halaman web yang akan di-scraper
url = 'https://bpjt.pu.go.id/tabel-tarif-tol'

# Buka halaman web
driver.get(url)

# Temukan elemen dropdown
dropdown = driver.find_element(By.ID, 'toll-roads')

# Buat objek Select dari elemen dropdown
select = Select(dropdown)

# ...

# Buat list kosong untuk menyimpan semua DataFrame
all_dataframes = []

# Buat list kosong untuk menyimpan nilai (value) dan teks (text) dari opsi (option)
option_values = []
option_texts = []

# Loop melalui semua opsi dalam dropdown
for option in select.options:
    option_value = option.get_attribute('value')
    option_text = option.text

    # Simpan nilai (value) dan teks (text) dari opsi (option)
    option_values.append(option_value)
    option_texts.append(option_text)

    # Pilih opsi dalam dropdown
    select.select_by_value(option_value)

    # Klik tombol "Lihat Tarif Tol"
    get_image_button = driver.find_element(By.ID, 'get-image')
    get_image_button.click()

    # Tunggu hingga tabel terisi (gunakan waktu tunggu jika diperlukan)
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.ID, 'tariffs-table')))

    # Dapatkan isi tabel menggunakan BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    kepmen = soup.find('div', {'id': 'kepmen'}).text
    table_data = soup.find('table', {'id': 'tariffs-table'})

    # Buat list kosong untuk menyimpan data dari satu elemen tabel
    data = []

    # Loop melalui baris dalam elemen <tbody> tabel
    for row in table_data.find('tbody').find_all('tr'):
        cells = row.find_all('td')
        if len(cells) == 8:
            asal = cells[0].text
            tujuan = cells[1].text
            golongan = [td.text for td in cells[2:]]
        else:
            tujuan = cells[0].text
            golongan = [td.text for td in cells[1:]]
        data.append([option_value, option_text, kepmen, asal, tujuan] + golongan)

    # Buat DataFrame untuk setiap elemen tabel
    df = pd.DataFrame(data, columns=['Value', 'Jalan Tol', 'Kepmen','Asal Perjalanan', 'Tujuan Perjalanan', 'Golongan I', 'Golongan II', 'Golongan III', 'Golongan IV', 'Golongan V', 'Golongan VI'])

    all_dataframes.append(df)

# Tutup WebDriver
driver.quit()

# Gabungkan semua DataFrame menjadi satu
final_dataframe = pd.concat(all_dataframes, ignore_index=True)

# Cetak DataFrame akhir
print(final_dataframe)

     Value                                 Jalan Tol  \
0       89                       Akses Tanjung Priok   
1      171               Bakauheni - Terbanggi Besar   
2      119  Bali Mandara (Nusa Dua-Ngurah Rai-Benoa)   
3      119  Bali Mandara (Nusa Dua-Ngurah Rai-Benoa)   
4      119  Bali Mandara (Nusa Dua-Ngurah Rai-Benoa)   
...    ...                                       ...   
1202   137            Ujung Pandang Seksi 1, 2 dan 3   
1203   137            Ujung Pandang Seksi 1, 2 dan 3   
1204   137            Ujung Pandang Seksi 1, 2 dan 3   
1205   137            Ujung Pandang Seksi 1, 2 dan 3   
1206   137            Ujung Pandang Seksi 1, 2 dan 3   

                                               Kepmen  \
0                                    1522/KPTS/M/2020   
1                                    1522/KPTS/M/2020   
2                                     455/KPTS/M/2023   
3                                     455/KPTS/M/2023   
4                                     455/

In [45]:
final_dataframe.head(10)

,Asal Perjalanan,Tujuan Perjalanan,Golongan I,Golongan II,Golongan III,Golongan IV,Golongan V,Golongan VI,Option Value,Option Text,Kepmen
0,"Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...","Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...",16000,23500,23500,31500,31500,-,89,Akses Tanjung Priok,1522/KPTS/M/2020
1,"Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...","Seksi E-1, E-2, E-2A, NS dan NS (Rorotan-Kebon...",16000,23500,23500,31500,31500,-,171,Bakauheni - Terbanggi Besar,1522/KPTS/M/2020
2,Pelabuhan Bakauheni,Bakauheni Selatan,6000,9000,9000,12000,12000,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
3,Pelabuhan Bakauheni,Bakauheni Utara,12000,18000,18000,24000,24000,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
4,Pelabuhan Bakauheni,Kalianda,37000,55500,55500,73500,73500,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
5,Pelabuhan Bakauheni,Lematang,100500,150500,150500,200500,200500,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
6,Pelabuhan Bakauheni,Kotabaru,106000,159000,159000,212000,212000,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
7,Pelabuhan Bakauheni,Natar,130500,195500,195500,260500,260500,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
8,Pelabuhan Bakauheni,Tegineneng Timur,146500,220000,22000,293000,293000,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023
9,Pelabuhan Bakauheni,Tegineneng Barat,146500,220000,220000,293000,293000,-,119,Bali Mandara (Nusa Dua-Ngurah Rai-Benoa),455/KPTS/M/2023


In [47]:
writer = pd.ExcelWriter('Scrap.xlsx', engine='xlsxwriter')

# write each dataframe to a different worksheet
final_dataframe.to_excel(writer, sheet_name='Data', index=False)
#df.to_excel(writer, sheet_name='Data Dump', index=False)

# save the Excel file
writer.save()